In [ ]:
!pip install pyspark

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Trabalho Pratico") \
    .getOrCreate()

spark.version

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
df_titles = spark.read.csv('/content/drive/MyDrive/MBA/title_basics.tsv', header=True, inferSchema=True, sep='\t')
df_ratings = spark.read.csv('/content/drive/MyDrive/MBA/title_ratings.tsv', header=True, inferSchema=True, sep='\t')

In [ ]:
df_titles.printSchema()

In [ ]:
df_ratings.printSchema()

In [ ]:
df_total = df_titles.join(df_ratings, df_ratings.tconst == df_titles.tconst, "left_outer")

In [ ]:
df_total.printSchema()

In [ ]:
df_total = df_total.withColumn('startYear',df_total.startYear.cast('int'))

In [ ]:
from pyspark.sql.functions import col
##Quantos filmes (incluindo os da televisão) foram lançados no ano de 2015?
df_total.groupBy('startYear', 'titleType').count().sort(col('startYear').desc()).where(col('startYear') == 2015).show()

In [ ]:
#Qual o gênero de títulos mais frequente?
df_total.groupBy('genres').count().sort(col('count').desc()).show()

In [ ]:
##Qual o gênero com a melhor nota média de títulos?
#df_total.groupBy('genres').agg({'averageRating':'avg'}).show()
df_total.groupBy('genres').avg('averageRating').sort(col('avg(averageRating)').desc()).show()

In [ ]:
##Qual o vídeo game do gênero aventura mais bem avaliado em 2020?
df_total.select('primaryTitle', 'averageRating') \
        .where((col('titleType') == 'videoGame') & (col('startYear') == 2020) & (col('genres').contains('Adventure'))) \
        .sort(col('averageRating').desc()).show()

In [ ]:
##Quantos títulos de filmes diferentes existem?
df_titles.select('primaryTitle').distinct().count()

In [ ]:
##Qual a duração média dos filmes com conteúdo adulto? Use uma combinação de filter() e describe().

df_total.filter(df_total.isAdult == '1').describe(['runtimeMinutes']).show()

In [ ]:
##Quantos filmes têm o título atual (“primary”) diferente do título original? Use uma combinação de filter, e count().

df_total.filter(df_total.primaryTitle != df_total.originalTitle).count()

In [ ]:
from pyspark.sql.functions import length
##Qual o filme que tem o nome mais longo?
df_titles.orderBy(length(col("primaryTitle")).desc()).show()

In [ ]:
## Qual filme tem a maior quantidade de votos? Dica: Use describe().
#df_total.where((df_total.titleType == 'movie') | (df_total.titleType == 'tvMovie')).sort(col('numVotes').desc()).show()
df_total.sort(col('numVotes').desc()).show()

In [ ]:
## Qual é a menor nota média de um filme? Use describe().
df_total.describe(['averageRating']).show()